In [2]:
import pandas as pd
import numpy as np
import re
import datetime as dt

In [19]:
# Adding KBM value to df
df = pd.read_csv('C:/Users/sgulbin/Work/Scoring (Accidents)/2020-03Test_v2/FTR_score/users_March.csv')
kbm = pd.read_csv('C:/Users/sgulbin/Work/Scoring (Accidents)/2020-03Test_v2/KBM_data/KBM_data_4_13_2020.csv')
df = pd.merge(df,kbm,on = 'user_id', how = 'left')
df['age'] = np.clip((pd.to_datetime('today')-pd.to_datetime(df.birthday, errors = 'coerce')).astype('timedelta64[Y]'),19.0,80.0)
df['exp'] = np.clip((pd.to_datetime('today')-pd.to_datetime(df.license_set_date, errors = 'coerce')).astype('timedelta64[Y]'),1.0,62.0)

In [20]:
# Generating synthetic sex
df['patronymic_name'] = df['patronymic_name'].fillna('Unknown')
df['patronymic_name_end'] = df['patronymic_name'].str.strip().str[-2:]
df.loc[df.patronymic_name_end.str.contains('ич'),'sex_synthetic'] = 'M'
df.loc[df.patronymic_name_end.str.contains('на'),'sex_synthetic'] = 'F'
df.loc[df.patronymic_name_end.str.contains('лы'),'sex_synthetic'] = 'M'
df.loc[df.patronymic_name_end.str.contains('зы'),'sex_synthetic'] = 'F'
df.loc[df.patronymic_name_end.str.contains('wn'),'sex_synthetic'] = 'Unknown'
df['sex_synthetic'] = df['sex_synthetic'].fillna('Unknown')
# print(df)

In [21]:
# Generating age-exp coefficients
K_age_exp_max = 1.2
K_age_exp_min = 0.725
K_intercept = -4.88430556234637
K_f_age = 1.26397937674965
K_f_exp = 0.0762464696907463
f_age_exp_min = K_f_age*np.log(19)+K_f_exp*np.log(1)+K_intercept
f_age_exp_max = K_f_age*np.log(50)+K_f_exp*np.log(20)+K_intercept
K = K_age_exp_max-((K_f_age*np.log(19)+K_f_exp*np.log(1)+K_intercept)-f_age_exp_min)*(K_age_exp_max-K_age_exp_min)/(f_age_exp_max-f_age_exp_min)
ages = []
exps = []
koefs = []
for age in range(19,101):
    for exp in range(1,83):
        K = np.clip(K_age_exp_max-((K_f_age*np.log(age)+K_f_exp*np.log(exp)+K_intercept)-f_age_exp_min)*(K_age_exp_max-K_age_exp_min)/(f_age_exp_max-f_age_exp_min),K_age_exp_min,K_age_exp_max)
        ages.append(age)
        exps.append(exp)
        koefs.append(K)
age_exp_df = pd.DataFrame(data = ({'age':ages, 'exp':exps, 'K_age_exp': koefs}))
# print(age_exp_df)

In [22]:
# Adding coefficients to df
df_K_sex = pd.DataFrame(data = ({'sex': ['M', 'F', 'Unknown'], 'K_sex': [1.0,1.10,1.0]}))
df_K_KBM = pd.DataFrame(data = ({'KBM': [0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95,1.0,1.4,1.55,2.3,2.45,np.nan],
                                 'K_KBM': [0.9,0.9,0.9,0.9,0.9,0.9,0.95,0.95,1.0,1.0,1.03,1.1,1.1,1.1,1.1,1.03]}))
df = pd.merge(df, df_K_sex, how = 'left', left_on = 'sex_synthetic', right_on = 'sex')
df = pd.merge(df, df_K_KBM, how = 'left', left_on = 'KBM', right_on = 'KBM')
df = pd.merge(df, age_exp_df, how = 'left', left_on = ['age','exp'], right_on = ['age','exp'])
df['K_age_exp'] = np.where(df['age'].isnull(), 1.0, np.where(df['exp'].isnull(),1.0,np.where((df['age']-df['exp'])<16,1.0,df['K_age_exp'])))
df['K_age_exp'] = df['K_age_exp'].fillna(1.0)
df['K_KBM'] = df['K_KBM'].fillna(1.0)

df['age-exp'] = df['age']-df['exp']
df['K_KBM_model_score'] = np.clip(df.K_sex*df.K_KBM*df.K_age_exp,0.7,1.3)
df['K_KBM_model_score_normalized'] = 100*(df['K_KBM_model_score']-df['K_KBM_model_score'].max())/(df['K_KBM_model_score'].min()-df['K_KBM_model_score'].max())
df['K_KBM_model_score_normalized'] = df['K_KBM_model_score_normalized'].round(0)
df_output = df[['user_id', 'K_KBM_model_score']]
print(df_output)

           user_id  K_KBM_model_score
0       1055251350           1.090414
1        795251628           0.871072
2         20509940           0.965171
3         19758603           0.714272
4         20878816           0.757266
5         20991334           0.799729
6         21259762           0.987710
7         55750352           1.058045
8         20749780           0.725286
9         21129863           0.700000
10       142751687           0.766965
11        20086426           0.876807
12        20042683           1.045000
13        20832888           0.839374
14        19651525           1.185910
15        19613964           0.703639
16       129500795           0.703197
17       420501817           1.161486
18        20840338           1.175543
19       439000681           0.930601
20      1078000813           1.157728
21        20061163           0.984771
22        20061181           0.837138
23       922001101           1.006086
24        21287187           1.196331
25       619

In [24]:
# df.to_csv('C:/Users/sgulbin/Work/Scoring (Accidents)/2020-03Test_v2/KBM_model_norm-ed_val.csv')
df_output.to_csv('C:/Users/sgulbin/Work/Scoring (Accidents)/2020-03Test_v2/FTR_score/FTR_score_for_March_users.csv')

In [23]:
test_age_exp = age_exp_df.loc[(age_exp_df.age == 28)&(age_exp_df.exp == 8)]
test_score = df.loc[(df.age == 28.0)&(df.exp == 8.0)&(df.K_sex == 1.0)&(df.K_KBM == 1.0)]
# print(test_score)
print(df.K_KBM_model_score.mean())

0.9669864030133709
